#### Saturday, February 10, 2024

How do I load a pdf file, and then extract the text from it?

#### Libraries to consider:

* [pypdf](https://github.com/py-pdf/PyPDF) 
* [pdfminer.six](https://github.com/pdfminer/pdfminer.six)
* [pdfplumber](https://github.com/jsvine/pdfplumber) 

In [1]:
import os 

pdfFolder = "../data"

In [2]:
def list_files_in_folder(folder_path):
    # Get the list of files in the specified folder
    files = os.listdir(folder_path)

    # Filter out subdirectories, leaving only files
    files = [f for f in files if os.path.isfile(os.path.join(folder_path, f))]

    return files

In [3]:
pdfFiles = list_files_in_folder(pdfFolder)

In [4]:
for file_name in pdfFiles:
    print(file_name)

Jordan B. Peterson -12 Rules for Life_ An Antidote to Chaos.pdf
Jordan Peterson - Beyond Order_ 12 More Rules For Life.pdf


#### pypdf

* https://github.com/py-pdf/PyPDF
* https://pypdf.readthedocs.io/en/latest/
* mamba install conda-forge::pypdf 

In [5]:
from pypdf import PdfReader

In [6]:
pdfFile = pdfFolder +  "/" + pdfFiles[1]
print(pdfFile)

../data/Jordan Peterson - Beyond Order_ 12 More Rules For Life.pdf


In [7]:
reader = PdfReader(pdfFile)

In [8]:
page = reader.pages[22]

In [9]:
print(page.extract_text())

      
D O  N O T  C A R E L E S S L Y  D E N I G R A T E  S O C I A L
I N S T I T U T I O N S  O R  C R E A T I V E  A C H I E V E M E N T
                        
For years, I saw a client who lived by himself. *  He was isolated in
many other ways in addition to his living situation. He had extremely
limited family ties. Both of his daughters had moved out of the
country, and did not maintain much contact, and he had no other
relatives except a father and sister from whom he was estranged. His
wife and the mother of his children had passed away years ago, and
the sole relationship he endeavored to establish while he saw me
over the course of more than a decade and a half terminated
tragically when his new partner was killed in an automobile accident.
When we began to work together, our conversations were
decidedly awkward. He was not accustomed to the subtleties of social
interaction, so his behaviors, verbal and nonverbal, lacked the dance-
like rhythm and harmony that characterize

In [10]:
page

{'/Type': '/Page',
 '/Annots': [IndirectObject(374, 0, 140537303314000)],
 '/Contents': {'/Filter': '/FlateDecode'},
 '/MediaBox': [0, 0, 612, 792],
 '/Parent': {'/Type': '/Pages',
  '/Count': 388,
  '/Kids': [IndirectObject(7871, 0, 140537303314000),
   IndirectObject(2, 0, 140537303314000),
   IndirectObject(34, 0, 140537303314000),
   IndirectObject(55, 0, 140537303314000),
   IndirectObject(87, 0, 140537303314000),
   IndirectObject(112, 0, 140537303314000),
   IndirectObject(129, 0, 140537303314000),
   IndirectObject(208, 0, 140537303314000),
   IndirectObject(224, 0, 140537303314000),
   IndirectObject(257, 0, 140537303314000),
   IndirectObject(287, 0, 140537303314000),
   IndirectObject(292, 0, 140537303314000),
   IndirectObject(295, 0, 140537303314000),
   IndirectObject(298, 0, 140537303314000),
   IndirectObject(300, 0, 140537303314000),
   IndirectObject(302, 0, 140537303314000),
   IndirectObject(304, 0, 140537303314000),
   IndirectObject(306, 0, 140537303314000),
   In

In [11]:
for page in reader.pages:
    if "/Annots" in page:
        for annot in page["/Annots"]:
            obj = annot.get_object()
            annotation = {"subtype": obj["/Subtype"], "location": obj["/Rect"]}
            print(annotation)

{'subtype': '/Link', 'location': [254.25003, 655.5, 357.75003, 666]}
{'subtype': '/Link', 'location': [283.50003, 422.99997, 414.00003, 433.49997]}
{'subtype': '/Link', 'location': [297.00003, 412.49997, 426.00003, 422.99997]}
{'subtype': '/Link', 'location': [77.250008, 562.5, 176.25002, 576]}
{'subtype': '/Link', 'location': [77.250008, 549, 326.25003, 562.5]}
{'subtype': '/Link', 'location': [77.250008, 535.5, 119.250015, 549]}
{'subtype': '/Link', 'location': [77.250008, 478.49997, 486.75003, 514.5]}
{'subtype': '/Link', 'location': [77.250008, 421.49997, 505.50006, 457.49997]}
{'subtype': '/Link', 'location': [77.250008, 382.49997, 373.50003, 400.49997]}
{'subtype': '/Link', 'location': [77.250008, 325.49997, 508.50006, 361.49997]}
{'subtype': '/Link', 'location': [77.250008, 286.49994, 279.00003, 304.49997]}
{'subtype': '/Link', 'location': [77.250008, 247.49994, 248.25003, 265.49994]}
{'subtype': '/Link', 'location': [77.250008, 190.49994, 506.25006, 226.49994]}
{'subtype': '/Li

{'subtype': '/Link', 'location': [119.250015, 678, 129.000015, 688.5]}
{'subtype': '/Link', 'location': [149.250015, 678, 163.500015, 688.5]}
{'subtype': '/Link', 'location': [168.00002, 678, 181.50002, 688.5]}
{'subtype': '/Link', 'location': [186.00002, 678, 201.00003, 688.5]}
{'subtype': '/Link', 'location': [161.250015, 667.5, 171.00002, 678]}
{'subtype': '/Link', 'location': [157.500015, 657, 171.75002, 667.5]}
{'subtype': '/Link', 'location': [123.750015, 646.5, 137.250015, 657]}
{'subtype': '/Link', 'location': [147.000015, 636, 159.750015, 646.5]}
{'subtype': '/Link', 'location': [108.750015, 625.5, 121.500015, 636]}
{'subtype': '/Link', 'location': [136.500015, 625.5, 150.000015, 636]}
{'subtype': '/Link', 'location': [154.500015, 625.5, 168.75002, 636]}
{'subtype': '/Link', 'location': [233.25003, 615, 247.50003, 625.5]}
{'subtype': '/Link', 'location': [165.750015, 604.5, 178.50002, 615]}
{'subtype': '/Link', 'location': [139.500015, 583.5, 153.750015, 594]}
{'subtype': '/Li

In [12]:
for page in reader.pages:
    if "/Annots" in page:
        for annot in page["/Annots"]:
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Text":
                print(annot.get_object()["/Contents"])

In [13]:
for page in reader.pages:
    if "/Annots" in page:
        for annot in page["/Annots"]:
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Highlight":
                coords = annot.get_object()["/QuadPoints"]
                x1, y1, x2, y2, x3, y3, x4, y4 = coords

In [14]:
page.extract_text(extraction_mode="layout", layout_mode_debug_path="debug2")

AttributeError: 'str' object has no attribute 'joinpath'

In [ ]:
# extract text in a fixed width format that closely adheres to the rendered
# layout in the source pdf
print(page.extract_text(extraction_mode="layout"))

*So that’s 1,000 pictures of paintings x 300 days per year x 4 = 1,200,000 paintings. That
has to be some sort of record (not that it matters, but it’s comical to consider), mostly
because I do not think it would have been possible to see that many paintings before
internet technology made massive databases possible.


In [ ]:
# extract text preserving horizontal positioning without excess vertical
# whitespace (removes blank and "whitespace only" lines)
print(page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False))

*So that’s 1,000 pictures of paintings x 300 days per year x 4 = 1,200,000 paintings. That
has to be some sort of record (not that it matters, but it’s comical to consider), mostly
because I do not think it would have been possible to see that many paintings before
internet technology made massive databases possible.


In [ ]:
# adjust horizontal spacing
print(page.extract_text(extraction_mode="layout", layout_mode_scale_weight=1.0))

*So that’s 1,000 pictures of paintings x 300 days per year x 4 = 1,200,000 paintings. That
has to be some sort of record (not that it matters, but it’s comical to consider), mostly
because I do not think it would have been possible to see that many paintings before
internet technology made massive databases possible.


In [ ]:
# exclude (default) or include (as shown below) text rotated w.r.t. the page
print(page.extract_text(extraction_mode="layout", layout_mode_strip_rotated=False))

*So that’s 1,000 pictures of paintings x 300 days per year x 4 = 1,200,000 paintings. That
has to be some sort of record (not that it matters, but it’s comical to consider), mostly
because I do not think it would have been possible to see that many paintings before
internet technology made massive databases possible.


In [ ]:
page = reader.pages[31]
print(page.extract_text())

and attractive to potential mates. He also wants to convince competitors for
that space to keep their distance.
Birds—and Territory
My dad and I designed a house for a wren family when I was ten years old. It
looked like a Conestoga wagon, and had a front entrance about the size of a
quarter. This made it a good house for wrens, who are tiny, and not so good
for other, larger birds, who couldn’t get in. My elderly neighbour had a
birdhouse, too, which we built for her at the same time, from an old rubber
boot. It had an opening large enough for a bird the size of a robin. She was
looking forward to the day it was occupied.
A wren soon discovered our birdhouse, and made himself at home there.
We could hear his lengthy, trilling song, repeated over and over, during the
early spring. Once he’d built his nest in the covered wagon, however, our
new avian tenant started carrying small sticks to our neighbour’s nearby boot.
He packed it so full that no other bird, large or small, could possib

#### pdf miner six

* https://github.com/pdfminer/pdfminer.six
* https://pdfminersix.readthedocs.io/en/latest/
* mamba install conda-forge::pdfminer.six

In [15]:
from pdfminer.high_level import extract_text

In [21]:
pdfFile

'../data/Jordan Peterson - Beyond Order_ 12 More Rules For Life.pdf'

In [16]:
text = extract_text(pdfFile)
# 7.8s

In [17]:
print(repr(text))

'\x0cA L S O   B Y   J O R D A N   B .   P E T E R S O N\n\nMaps of Meaning: The Architecture of Belief\n\n12 Rules for Life: An Antidote to Chaos\n\n\x0c\x0cPORTFOLIO / PENGUIN\nAn imprint of Penguin Random House LLC\npenguinrandomhouse.com\n\nCopyright © 2021 by Dr. Jordan B. Peterson\nPenguin supports copyright. Copyright fuels creativity, encourages diverse voices, promotes free speech, and creates a\nvibrant culture. Thank you for buying an authorized edition of this book and for complying with copyright laws by not\nreproducing, scanning, or distributing any part of it in any form without permission. You are supporting writers and\nallowing Penguin to continue to publish books for every reader.\n\nISBN 9780593420164 (export edition)\n\n\x00\x00\x00\x00\x00\x00\x00 \x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00-\x00\x00-\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\n\nNames: Peterson, Jordan B., author.\nTitle: Beyond order : 12 

In [18]:
print(text)

A L S O   B Y   J O R D A N   B .   P E T E R S O N

Maps of Meaning: The Architecture of Belief

12 Rules for Life: An Antidote to Chaos

PORTFOLIO / PENGUIN
An imprint of Penguin Random House LLC
penguinrandomhouse.com

Copyright © 2021 by Dr. Jordan B. Peterson
Penguin supports copyright. Copyright fuels creativity, encourages diverse voices, promotes free speech, and creates a
vibrant culture. Thank you for buying an authorized edition of this book and for complying with copyright laws by not
reproducing, scanning, or distributing any part of it in any form without permission. You are supporting writers and
allowing Penguin to continue to publish books for every reader.

ISBN 9780593420164 (export edition)

                              -  -                

Names: Peterson, Jordan B., author.
Title: Beyond order : 12 more rules for life / Jordan B. Peterson.
Description: [New York, New York] : Penguin ; Portfolio, [2021] | Includes
bibliographical references and index.
Identifi

In [19]:
from pdfminer.high_level import extract_pages
for page_layout in extract_pages(pdfFile):
    for element in page_layout:
        print(element)
        
# 8.2s

<LTFigure(Image0) 0.000,0.000,612.000,792.000 matrix=[612.00,0.00,0.00,792.00, (0.00,0.00)]>
<LTTextBoxHorizontal(0) 201.023,661.284,409.482,672.534 'A L S O   B Y   J O R D A N   B .   P E T E R S O N\n'>
<LTTextBoxHorizontal(1) 194.461,630.534,417.527,641.784 'Maps of Meaning: The Architecture of Belief\n'>
<LTTextBoxHorizontal(2) 208.805,611.784,403.204,623.034 '12 Rules for Life: An Antidote to Chaos\n'>
<LTRect 72.000,3.000,540.000,720.000>
<LTRect 72.000,3.000,540.000,720.000>
<LTFigure(X4) 107.250,104.250,504.750,720.000 matrix=[397.50,0.00,0.00,615.75, (107.25,104.25)]>
<LTTextBoxHorizontal(0) 222.516,655.833,389.490,685.578 'PORTFOLIO / PENGUIN\nAn imprint of Penguin Random House LLC\npenguinrandomhouse.com\n'>
<LTTextBoxHorizontal(1) 79.395,570.333,532.614,621.078 'Copyright © 2021 by Dr. Jordan B. Peterson\nPenguin supports copyright. Copyright fuels creativity, encourages diverse voices, promotes free speech, and creates a\nvibrant culture. Thank you for buying an authorize

In [20]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
for page_layout in extract_pages(pdfFile):
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            print(element.get_text())

A L S O   B Y   J O R D A N   B .   P E T E R S O N

Maps of Meaning: The Architecture of Belief

12 Rules for Life: An Antidote to Chaos

PORTFOLIO / PENGUIN
An imprint of Penguin Random House LLC
penguinrandomhouse.com

Copyright © 2021 by Dr. Jordan B. Peterson
Penguin supports copyright. Copyright fuels creativity, encourages diverse voices, promotes free speech, and creates a
vibrant culture. Thank you for buying an authorized edition of this book and for complying with copyright laws by not
reproducing, scanning, or distributing any part of it in any form without permission. You are supporting writers and
allowing Penguin to continue to publish books for every reader.

ISBN 9780593420164 (export edition)

                              -  -                

Names: Peterson, Jordan B., author.
Title: Beyond order : 12 more rules for life / Jordan B. Peterson.
Description: [New York, New York] : Penguin ; Portfolio, [2021] | Includes
bibliographical references and index.
Identifiers